# Retrieval-Augmented Generation (RAG)

Install the Hugging Face libraries to run this notebook.

In [98]:
!pip install transformers wikipedia

Defaulting to user installation because normal site-packages is not writeable


In [99]:
import torch
import torch.nn.functional as F

import wikipedia
import json

In [100]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Document ingestion

In [101]:
def extract_wikipedia_pages(page_titles):
    """
    Extracts Wikipedia pages and stores them in a dictionary.

    Args:
        page_titles: A list of Wikipedia page titles to extract.

    Returns:
        A dictionary containing the text of each Wikipedia page.
    """

    page_data = {}
    for title in page_titles:
        try:
            page = wikipedia.page(title)
            content = page.content.strip()
            content = content.replace("\n", "")
            page_data[page.title] = content
        except wikipedia.exceptions.PageError:
            print(f"Page '{title}' not found.")
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Disambiguation error for '{title}': {e.options}")

    return page_data

In [102]:
page_titles = [
               "Roger Apéry",
               "Owen Willans Richardson",
               "Otto Sackur",
               "Ludvig Lorenz",
               "Klaus von Klitzing",
               "Henri Victor Regnault",
               "Erwin Madelung",
              ]

# Uncomment the next line to scroll through Wikipedia
# wikipedia_data = extract_wikipedia_pages(page_titles)

Save the dictionary using `json.dump()`:

In [103]:
# with open('wikipedia_data.json', 'w') as f:
#     json.dump(wikipedia_data, f, indent=4)

Load the dictionary using `json.load()`:

In [104]:
with open('wikipedia_data.json', 'r') as f:
    wikipedia_data = json.load(f)

In [105]:
for doc in wikipedia_data:
    print(len(wikipedia_data[doc]))

3107
3455
1683
1873
1762
3431
1487


## Document pre-processing

We load just the tokenizer:

In [106]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model_max_length = tokenizer.model_max_length
model_max_length

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

512

In [107]:
encoded_text = tokenizer.encode(["hello", "how are you?"])
tokenizer.decode(encoded_text)

'[CLS] hello [SEP] how are you? [SEP]'

Naive version:

In [108]:
def text_splitting(text, chunk_length = 300, chunk_overlap = 100):
    out = []
    for i in range(0,len(text), chunk_length - chunk_overlap):
        out.append(text[i: i + chunk_length])
    return out

text_splitting("".join([str(x) for x in range(20)]), 5, 2)

['01234',
 '34567',
 '67891',
 '91011',
 '11121',
 '21314',
 '14151',
 '51617',
 '17181',
 '819']

In [109]:
def text_splitting_paragraph(text, chunk_length = 300):
    out = []
    paragraph_list = text.split(".")
    current_text = ""
    length = 0
    for par in paragraph_list:
        if length != 0 and length + len(par) < chunk_length:
            current_text += ". " + par
            length += len(par)
        else:
            if len(current_text) != 0:
                out.append(current_text + ".")
            current_text = par
            length = len(par)
    return out

In [110]:
wikipedia_data_splits = {}

for doc in wikipedia_data.keys():
    wikipedia_data_splits[doc] = text_splitting_paragraph(wikipedia_data[doc])

first_key = page_titles[0]
wikipedia_data_splits[first_key][:3]

["Roger Apéry (French: [apeʁi]; 14 November 1916, Rouen – 18 December 1994, Caen) was a French mathematician most remembered for Apéry's theorem, which states that ζ(3) is an irrational number.  Here, ζ(s) denotes the Riemann zeta function.",
 '== Biography ==Apéry was born in Rouen in 1916 to a French mother and Greek father.  His childhood was spent in Lille until 1926, when the family moved to Paris, where he studied at the Lycée Ledru-Rollin and the Lycée Louis-le-Grand.   He was admitted  at the École normale supérieure in 1935.',
 '  His studies were interrupted at the start of World War II; he was mobilized in September 1939, taken prisoner of war in June 1940, repatriated with pleurisy in June 1941, and hospitalized until August 1941.']

In [111]:
min_doc = min(len(wikipedia_data_splits[doc]) for doc in wikipedia_data_splits)
max_doc = max(len(wikipedia_data_splits[doc]) for doc in wikipedia_data_splits)
av_doc = sum(len(wikipedia_data_splits[doc]) for doc in wikipedia_data_splits) / len(wikipedia_data_splits)

min_doc,max_doc,av_doc

(6, 14, 9.571428571428571)

## Generating embeddings

Now we load the embedder:

In [112]:
from transformers import AutoModel

model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)


In [113]:
inputs = tokenizer("Hello, world!", return_tensors="pt")
outputs = model(**inputs)

output_dim = outputs.last_hidden_state.size(2)
output_dim

384

The embedder needs to know whether the document is a document or a query.

In [114]:
def embed(chunk_list, doc_type="document"):
    encoded_docs = tokenizer(["search_{}: {}".format(doc_type, chunk) for chunk in chunk_list],
                                 padding = True,
                                 return_tensors="pt")
    output = model(**encoded_docs) # (batch, input_length, output_dim)
    token_embeddings = output.last_hidden_state
    output_embeddings = torch.sum(token_embeddings, 1)
    output_embeddings = F.normalize(output_embeddings, p=2, dim=1)
    return output_embeddings # (batch, output_dim)

In [115]:
embed(["hello", "another document", "and another one"]).shape

torch.Size([3, 384])

In [125]:
def populate_database(dic_splits, batch_size = 1):
    n_chunks = sum([len(dic_splits[doc]) for doc in dic_splits])
    vectorial_database = torch.zeros([n_chunks, output_dim], requires_grad = False).to(device)
    chunk_list = []
    n = 0
    for doc in dic_splits.keys():
        chunk_list_doc = dic_splits[doc]
        print(doc, len(chunk_list_doc))
        for i in range(0, len(chunk_list_doc), batch_size):
            batch = chunk_list_doc[i:i+batch_size]
            chunk_list += batch
            embeddings = embed(batch, doc_type = "document")
            vectorial_database[n:n+len(batch)] = embeddings
            n += len(batch)
    return chunk_list, vectorial_database

# Uncomment this to populate the database
chunk_list, vectorial_database = populate_database(wikipedia_data_splits)

Roger Apéry 12
Owen Richardson 14
Otto Sackur 6
Ludvig Lorenz 8
Klaus von Klitzing 7
Henri Victor Regnault 14
Erwin Madelung 6


Save the vectorial database using `torch.save()`:

In [126]:
torch.save(vectorial_database, 'vectorial_database.pth')

with open('chunk_list.json', 'w') as f:
    json.dump(chunk_list, f, indent=4)

Load the database using `torch.load()`:

In [127]:
vectorial_database = torch.load('vectorial_database.pth')
vectorial_database.to(device)
vectorial_database.requires_grad_(False)

with open('chunk_list.json', 'r') as f:
    chunk_list = json.load(f)

In [128]:
len(chunk_list), vectorial_database.shape

(67, torch.Size([67, 384]))

In [129]:
for i, embedding_vector in enumerate(vectorial_database[:20]):
    print(embedding_vector[:5], chunk_list[i][:50])

tensor([-0.0557,  0.0384, -0.0307, -0.0143, -0.0188]) Roger Apéry (French: [apeʁi]; 14 November 1916, Ro
tensor([ 0.0187,  0.0690, -0.0273, -0.0288,  0.0369]) == Biography ==Apéry was born in Rouen in 1916 to 
tensor([ 0.0271,  0.0228, -0.0974,  0.0471, -0.0042])   His studies were interrupted at the start of Wor
tensor([-0.0875,  0.0111, -0.0129, -0.0130, -0.1274])  He wrote his doctoral thesis in algebraic geometr
tensor([-0.0636,  0.0460, -0.0357,  0.1095, -0.0656])  In 1949 he was appointed Professor at the Univers
tensor([-0.1005,  0.0247, -0.0105,  0.1190,  0.0525])  An indication of the difficulty is that the corre
tensor([-0.1744,  0.0473,  0.0322,  0.0616,  0.0066])  Nevertheless, many mathematicians have since work
tensor([ 0.0378,  0.0296, -0.0356,  0.0142,  0.0272]) Apéry was active in politics and for a few years i
tensor([-0.0365,  0.0051,  0.0233,  0.0197, -0.0063]) == Personal life ==Apéry married in 1947 and had t
tensor([-0.0262,  0.0947, -0.0276, -0.0283, -0.0226])  

## Retrieval

In [130]:
def similarity(query_embeddings, doc_embeddings):
    return query_embeddings @ doc_embeddings.T

In [131]:
query_embeddings = embed([
    "What is TSNE?",
    "Who is Laurens van der Maaten?",
], "query")

doc_embeddings = embed([
    "TSNE is a dimensionality reduction algorithm created by Laurens van Der Maaten",
], "document")

with torch.no_grad():
    print(similarity(query_embeddings, doc_embeddings))

tensor([[0.6542],
        [0.4483]])


In [132]:
def retrieve(query, 
             vectorial_database = vectorial_database, 
             chunk_list = chunk_list, 
             topk = 5,
             verbose = False):
    query_embedding = embed([query], "query")
    query_embedding.to(device)
    similarity_scores = similarity(query_embedding, vectorial_database)
    topk = torch.topk(similarity_scores, k = topk)

    if verbose:
        for score, idx in zip(topk.values[0], topk.indices[0]):
            print(f"Score: {score:.4f}\nText:\n", chunk_list[idx], "\n")
    return "\n".join([chunk_list[i] for i in topk.indices[0].tolist()])

In [133]:
%%timeit
retrieve("What did Erwin Madelung study?")

7.57 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Alternative retrieval: SVM

In [134]:
import numpy as np
from sklearn import svm

def retrieve_SVM(query, 
             vectorial_database = vectorial_database, 
             chunk_list = chunk_list, 
             topk = 5):
    query_embedding = embed([query], "query")
    x = np.concatenate([query_embedding.detach().numpy(), vectorial_database.detach().numpy()])
    y = np.zeros(vectorial_database.size(0) + 1)
    y[0] = 1 # we have a single positive example

    clf = svm.LinearSVC(class_weight='balanced', verbose=False, max_iter=10000, tol=1e-6, C=0.1, dual="auto")
    clf.fit(x, y)
    similarities = clf.decision_function(x)
    sorted_ix = np.argsort(-similarities)
    for k in sorted_ix[1:topk+1]:
        print(f"Score: {similarities[k]:.4f}\nText:\n", chunk_list[k-1], "\n")
    return "\n".join([chunk_list[k-1] for k in sorted_ix[1:topk+1]])

In [135]:
retrieve_SVM("What did Erwin Madelung study?")

Score: -0.2089
Text:
 Erwin Madelung (18 May 1881 – 1 August 1972) was a German physicist. He was born in 1881 in Bonn.  His father was the surgeon Otto Wilhelm Madelung.  He earned a doctorate in 1905 from the University of Göttingen, specializing in crystal structure, and eventually became a professor. 

Score: -0.3036
Text:
                     1        +                              1            8                          +                              1            27                          +                              1            64                          +        ⋯        ≠                              p            q                                {\displaystyle 1+{\frac {1}{8}}+{\frac {1}{27}}+{\frac {1}{64}}+\cdots \neq {\frac {p}{q}}}  == See also ==Apéry's constantBasel problem== External links ==Apéry, François (1996). 

Score: -0.3241
Text:
  He specialized in atomic physics and quantum mechanics, and it was during this time he developed the Madelung equations, an al

"Erwin Madelung (18 May 1881 – 1 August 1972) was a German physicist. He was born in 1881 in Bonn.  His father was the surgeon Otto Wilhelm Madelung.  He earned a doctorate in 1905 from the University of Göttingen, specializing in crystal structure, and eventually became a professor.\n                    1        +                              1            8                          +                              1            27                          +                              1            64                          +        ⋯        ≠                              p            q                                {\\displaystyle 1+{\\frac {1}{8}}+{\\frac {1}{27}}+{\\frac {1}{64}}+\\cdots \\neq {\\frac {p}{q}}}  == See also ==Apéry's constantBasel problem== External links ==Apéry, François (1996).\n He specialized in atomic physics and quantum mechanics, and it was during this time he developed the Madelung equations, an alternative form of the Schrödinger equation.\n An indication 

## Full pipeline

This model does **extractive** question answering, meaning it can only points to the answer in the provided context.

In [136]:
from transformers import AutoModelForQuestionAnswering, pipeline

model_name = "deepset/tinyroberta-squad2"

QA = pipeline('question-answering', model=model_name, tokenizer=model_name, device=device)

Device set to use cpu


In [137]:
def query(prompt):
    topk_chunks = retrieve(prompt)
#     topk_chunks = retrieve_SVM(prompt)
    return QA(question=prompt, context=topk_chunks)

In [138]:
query("What did Erwin Madelung study?")

{'score': 0.6069597005844116,
 'start': 303,
 'end': 339,
 'answer': 'atomic physics and quantum mechanics'}